extract and create comparision report

In [1]:
import os
import json
import pandas as pd

# ─── CONFIGURE THESE ─────────────────────────────────────────────────────────
# Path under which each tool has its own subfolder.
BASE_DIR = r"C:\Users\mf23-4\Desktop\Smartbugs\results"
# Names of the tool-folders you want to scan:
TOOLS = [
    "mythril-0.24.7",
    "manticore-0.3.7",
    "confuzzius",
    "conkas",
    "osiris",
    "oyente",
    "sfuzz",
    "slither-0.10.4",
    "smartcheck"
]
# ───────────────────────────────────────────────────────────────────────────────

# Data collectors
summary = []
vuln_rows = []

total_contracts = 0
empty_tools = 0

for tool in TOOLS:
    tool_path = os.path.join(BASE_DIR, tool)
    if not os.path.isdir(tool_path):
        # tool folder missing or not a dir
        summary.append((tool, 0, 0))
        empty_tools += 1
        continue

    # find all subdirs (each named by a contract address)
    addrs = [d for d in os.listdir(tool_path)
             if os.path.isdir(os.path.join(tool_path, d))]
    if not addrs:
        summary.append((tool, 0, 0))
        empty_tools += 1
        continue

    valid = 0
    for addr in addrs:
        total_contracts += 1
        json_path = os.path.join(tool_path, addr, "result.json")
        if not os.path.isfile(json_path):
            continue
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        findings = data.get("findings", [])
        if findings:
            valid += 1
            # collect vulnerabilities
            for fnd in findings:
                vuln_rows.append({
                    "tool": tool,
                    "contract": addr,
                    "vuln_name": fnd.get("name"),
                    "severity": fnd.get("severity")
                })

    summary.append((tool, len(addrs), valid))

# Build DataFrames
df_summary = pd.DataFrame(summary,
                          columns=["tool", "n_addresses", "n_valid_jsons"])
df_vulns   = pd.DataFrame(vuln_rows,
                          columns=["tool", "contract", "vuln_name", "severity"])

# Overall stats
print(f"► Total contract addresses scanned: {total_contracts}")
print(f"► Number of tools with zero addresses: {empty_tools}")
print()

# Per-tool summary
print("Per-tool scan vs valid JSONs:")
print(df_summary.to_string(index=False))
print()

# Vulnerability types per tool
vuln_counts = (
    df_vulns
    .groupby(["tool", "vuln_name"])
    .size()
    .reset_index(name="count")
    .sort_values(["tool", "count"], ascending=[True, False])
)
print("Vulnerability types detected (per tool):")
print(vuln_counts.to_string(index=False))
print()

# Optionally, you can also pivot for a quick overview:
pivot = df_vulns.pivot_table(
    index="vuln_name", columns="tool", aggfunc="size", fill_value=0
)
print("Pivot table of vuln counts:")
print(pivot)


► Total contract addresses scanned: 5109
► Number of tools with zero addresses: 0

Per-tool scan vs valid JSONs:
           tool  n_addresses  n_valid_jsons
 mythril-0.24.7          557            137
manticore-0.3.7          547              0
     confuzzius          589            209
         conkas          600            339
         osiris          568            189
         oyente          582            254
          sfuzz          555              0
 slither-0.10.4          536            468
     smartcheck          575            563

Vulnerability types detected (per tool):
          tool                                                vuln_name  count
    confuzzius                                        Assertion Failure    268
    confuzzius                                         Block Dependency    237
    confuzzius                                         Integer Overflow    146
    confuzzius                             Transaction Order Dependency     31
    confuz